In [2]:
import pymysql
import pandas as pd
connection1 = pymysql.connect(host = '119.74.24.181', user = 'htx', password = 'Police123456', database = 'ASTRO')
connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')

df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection1)
df

C:\Users\gengy\AppData\Local\Temp\ipykernel_35788\2509261158.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query("SELECT * FROM astro.scam_management_system", connection1)


,report_number,initial_report_number,classification,scam_type,account_bank,account_number,account_phone_number,amount_scammed,amount_transcated,whatsapp_number,...,datetime_bank_account_frozen,date_bank_details_received,account_holder_name,account_holder_nric,acount_holder_phone_number,latest_balance_seized,is_coporate_account,referral_date,division,account_or_paynowlah_number
0,G/20201126/2064,None,Cheating,Job Scam,HSBC,1.49486E+11,86787284,14920.0,NaN,85199976,...,2022-02-25,2022-03-27,KMWPTSBDEMQMEC,x,96466392,4.03,0,2021-12-31,G,1.49486E+11
1,D/20110219/9828,None,Cheating,Loan Scam,HSBC,1.73329E+11,,22500.0,NaN,None,...,2022-01-03,2022-04-27,TRHNWFHFLOWPSR,x,91518420,3.76,0,2021-12-31,E,1.73329E+11
2,B/20150715/9514,None,Cheating,Job Scam,HSBC,1.73511E+11,,6631.0,NaN,82093039,...,2022-01-03,2022-04-28,IQCKSJHKIPSWEL,x,98911687,2.25,0,2021-12-31,E,1.73511E+11
3,A/20180509/4546,None,Cheating,Job Scam,HSBC,1.73516E+11,,0.0,NaN,81564132,...,2022-01-03,2022-04-28,UESRHHIDPYOTPT,x,98635049,2.25,0,2021-12-31,G,1.73516E+11
4,C/20110220/9485,None,Cheating,Job Scam,HSBC,1.73484E+11,,0.0,NaN,None,...,2022-01-03,2022-04-28,KBDHIIQSDDIEUJ,x,91815925,2.25,0,2021-12-31,G,1.73484E+11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83896,D/20230501/4291,ASC-pending SOA,Cheating,Job Scam,UOB,7798633162,,5000.0,0.0,None,...,2023-03-22,None,HMGLPYCRFVPCNJ,x,,NaN,0,2023-03-22,E,7798633162
83897,D/20180624/9648,ASC-pending SOA,Cheating,Job Scam,UOB,7693984523,,200.0,0.0,None,...,2023-03-23,None,SLDXDNXXCTRUVD,x,,NaN,0,2023-03-22,J,7693984523
83898,G/20100725/3779,ASC-pending SOA,Cheating,Job Scam,UOB,7695289890,,4950.0,0.0,None,...,2023-03-23,None,CTUUVAWOOHGDTC,x,,NaN,0,2023-03-22,D,7695289890
83899,F/20180623/4801,ASC-pending SOA,Cheating,Job Scam,UOB,7797948598,94362026,52.0,0.0,94041108,...,2023-03-23,None,CCDVQDUVWRAKFV,x,,NaN,0,2023-03-23,F,7797948598


In [17]:
from nicegui import ui
import pymysql
import pandas as pd
from test2 import *


connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')

cursor = connection2.cursor()
cursor.execute("SELECT UPDATE_TIME FROM information_schema.tables WHERE TABLE_SCHEMA = 'sys' AND TABLE_NAME = 'sys_config'")
result = cursor.fetchone()
initial_updated_time = result[0]


def data():
    connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')
    
    df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)
    
    return df_test
df = data()
print(df)


                               variable  value            set_time set_by  \
0          diagnostics.allow_i_s_tables    OFF 2023-03-31 14:54:33   None   
1               diagnostics.include_raw    OFF 2023-03-31 14:54:33   None   
2                                   HTX   None 2023-05-23 21:03:42   None   
3         ps_thread_trx_info.max_length  65535 2023-03-31 14:54:33   None   
4  statement_performance_analyzer.limit    100 2023-03-31 14:54:33   None   
5   statement_performance_analyzer.view   None 2023-03-31 14:54:33   None   
6                statement_truncate_len     64 2023-03-31 14:54:33   None   

  date_assigned  
0          None  
1          None  
2          None  
3          None  
4          None  
5          None  
6          None  


C:\Users\gengy\AppData\Local\Temp\ipykernel_6888\1891766276.py:18: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


In [21]:
df_scam_type = df[['scam_type', 'amount_scammed']].copy()
df_scam_type['scam_type'] = df_scam_type['scam_type'].str.lower()
df_scam_type['scam_type'] = df_scam_type['scam_type'].replace('loan scan', 'loan scam')

#   Group data according to scam type & the amount scammed for the respective scam types
df_scam_type = df_scam_type.groupby('scam_type').agg({'scam_type': 'count', 'amount_scammed': 'sum'})
df_scam_type.columns = ['num_reports', 'total_amount_scammed']
df_scam_type = df_scam_type.rename_axis('scam_type').reset_index()
df_scam_type['scam_type'] = df_scam_type['scam_type'].str.title()
df_scam_type['scam_type'] = df_scam_type['scam_type'].replace({
    'Bank Phishing Sms Scam': 'Bank Phishing SMS Scam',
    'Bec Scam': 'BEC Scam',
    'Cois': 'COIS',
    'Fgps': 'FGPS',
    'Gois': 'GOIS',
    'Osss': 'OSSS',
    'Non-Bank Phishing Sms Scam': 'Non-Bank Phishing SMS Scam'})

#   Sort the scam types according to their no. of reports, in descending order
df_scam_type = df_scam_type.sort_values(by = 'num_reports', ascending = False)

df_scam_type['total_amount_scammed'] = df_scam_type['total_amount_scammed'] / 1000000

df_scam_type

,scam_type,num_reports,total_amount_scammed
12,Job Scam,39664,155.200034
11,Investment Scam,10806,127.564759
3,E-Commerce Scam,7473,10.309434
7,Friend Impersonation Scam,4298,11.304567
21,Other Scam,3769,25.634940
24,Rental Scam,2788,8.254547
8,GOIS,2664,55.099198
13,Loan Scam,2161,9.010397
16,Love/Parcel Scam,1821,20.320972
25,S/M Impersonation Scam,1507,5.153877


In [6]:
num_train = 60000
print('num_train = len(train_data) ==> ', num_train)
indices = list(range(num_train))

print('len(indices) ==>', len(indices))
print(indices)

num_train = len(train_data) ==>  60000
len(indices) ==> 60000
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 20

In [7]:
import numpy as np
np.random.shuffle(indices)
print(indices)

[57688, 37673, 41357, 44890, 56817, 10785, 21587, 56170, 28523, 38994, 20948, 58861, 8759, 44115, 57395, 22288, 10136, 4810, 39648, 2479, 21311, 9558, 49668, 37323, 51338, 14271, 58627, 14386, 9191, 33219, 12881, 16465, 35163, 37374, 10938, 21527, 49148, 49251, 45248, 21684, 45159, 20159, 50, 55613, 13511, 36658, 38569, 28500, 26215, 50501, 12485, 14875, 58179, 13767, 24894, 13709, 13638, 18863, 35722, 2551, 35841, 42692, 23765, 27620, 14002, 20524, 19558, 19124, 7691, 43997, 5440, 14208, 19673, 57682, 26921, 35506, 3920, 31742, 294, 4230, 15065, 27332, 15770, 35923, 13809, 11929, 50170, 43038, 1305, 392, 20183, 19119, 16195, 55465, 32735, 45507, 37545, 48196, 25340, 55785, 49642, 31667, 5197, 35631, 33448, 33179, 52863, 57993, 39814, 6349, 16758, 49144, 32408, 46035, 27282, 28901, 48860, 51274, 992, 58545, 364, 4729, 34218, 41537, 40368, 25102, 32816, 21447, 42540, 54072, 45019, 11560, 48653, 27240, 41967, 5697, 15793, 50947, 13303, 37269, 2681, 25126, 8285, 17648, 6500, 8563, 36411, 

In [9]:
valid_size = 0.2 # This means the data set is divided into two parts where 80% for training 20% for validation 
split = int(np.floor(valid_size * num_train))
print(split)
train_idx, valid_idx = indices[split:], indices[:split]

print('len(train_idx) ==> ', len(train_idx))
print('len(valid_idx) ==> ', len(valid_idx))
print(train_idx)
print(valid_idx)

12000
len(train_idx) ==>  48000
len(valid_idx) ==>  12000
[2577, 23045, 1731, 53936, 16223, 48195, 12703, 23013, 27984, 14692, 33417, 28164, 36918, 28546, 9476, 41177, 22781, 39516, 1395, 21721, 32461, 10880, 12099, 37583, 49198, 32694, 53865, 25982, 23870, 53915, 43077, 59002, 13679, 33129, 22824, 36354, 29786, 40256, 53236, 3748, 20139, 43699, 53201, 52409, 34399, 20680, 42934, 38263, 19171, 16214, 11797, 10572, 47767, 27996, 56970, 53479, 17139, 5122, 36957, 44318, 26483, 34246, 18810, 22963, 12876, 1269, 33334, 22569, 8139, 59783, 1067, 26818, 26035, 35060, 44463, 34741, 18351, 43124, 39041, 47042, 34960, 45392, 52061, 13396, 3204, 34701, 54854, 38843, 31500, 58936, 4318, 24270, 18286, 16080, 25749, 2804, 41375, 49356, 5708, 56309, 5905, 44503, 7114, 28657, 52807, 1184, 57331, 59797, 5978, 45427, 22401, 17021, 12692, 33391, 1535, 58994, 20179, 9629, 33216, 54673, 59786, 11970, 12720, 9697, 27392, 18622, 19970, 5534, 35458, 27476, 48312, 26136, 20352, 23856, 42512, 15548, 18239, 328

In [69]:
import pymysql
import pandas as pd
connection2 = pymysql.connect(host = 'localhost', user = 'root', password = 'X-rayisharmful01', database = 'sys')

df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


df_test2 = df_test['variable'].value_counts().reset_index()

# rename columns
df_test2.columns = ['variable', 'count']
x_axis = df_test2['variable'].tolist()
y_axis = df_test2['count'].tolist()


C:\Users\user\AppData\Local\Temp\ipykernel_9596\1866772294.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_test = pd.read_sql_query("SELECT * FROM sys.sys_config", connection2)


['diagnostics.allow_i_s_tables',
 'diagnostics.include_raw',
 'ps_thread_trx_info.max_length',
 'statement_performance_analyzer.limit',
 'statement_performance_analyzer.view',
 'statement_truncate_len',
 'test',
 'testing']

In [57]:
telco_count_series = telco_df.groupby('telco').size()
telco_name_list = telco_count_series.index.to_list()
telco_count_list = telco_count_series.to_list()
telco_count_series[0] = 37
telco_count_series

telco
M1            37
MyRepublic    84
Singtel       25
Starhub        7
dtype: int64

In [23]:
df_scam_type_interim = df[['scam_type', 'amount_scammed']].copy()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].str.lower()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].replace('loan scan', 'loan scam')

df_scam_type_interim = df_scam_type_interim.groupby('scam_type').agg({'scam_type': 'count', 'amount_scammed': 'sum'})
df_scam_type_interim.columns = ['num_reports', 'total_amount_scammed']
df_scam_type_interim = df_scam_type_interim.rename_axis('scam_type').reset_index()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].str.title()
df_scam_type_interim['scam_type'] = df_scam_type_interim['scam_type'].replace({
        'Bank Phishing Sms Scam': 'Bank Phishing SMS Scam',
        'Bec Scam': 'BEC Scam',
        'Cois': 'COIS',
        'Fgps': 'FGPS',
        'Gois': 'GOIS',
        'Osss': 'OSSS',
        'Non-Bank Phishing Sms Scam': 'Non-Bank Phishing SMS Scam'})


df_scam_type_interim = df_scam_type_interim.sort_values(by = 'num_reports', ascending = False)
df_scam_type_interim
# top_scam_type_list = df_scam_type_interim.iloc[0:10]['scam_type'].values.tolist()

# #   Data Cleaning
# df_scam_type = df[['date_assigned', 'scam_type']].copy()
# df_scam_type = df_scam_type.dropna(subset=['date_assigned', 'scam_type'])
# df_scam_type['scam_type'] = df_scam_type['scam_type'].str.lower()
# df_scam_type['scam_type'] = df_scam_type['scam_type'].replace('loan scan', 'loan scam')
# df_scam_type['scam_type'] = df_scam_type['scam_type'].str.title()

# df_scam_type = df_scam_type[df_scam_type["scam_type"].isin(top_scam_type_list)].reset_index()

# df_scam_type['date_assigned'] = pd.to_datetime(df_scam_type['date_assigned'])
# counts = df_scam_type.groupby(['scam_type', 'date_assigned']).size()
# df_scam_type = counts.reset_index(name='count')
# df_scam_type
# pd.set_option('display.max_rows', 500)


,scam_type,num_reports,total_amount_scammed
12,Job Scam,39664,1.552000e+08
11,Investment Scam,10806,1.275648e+08
3,E-Commerce Scam,7473,1.030943e+07
7,Friend Impersonation Scam,4298,1.130457e+07
21,Other Scam,3769,2.563494e+07
24,Rental Scam,2788,8.254547e+06
8,GOIS,2664,5.509920e+07
13,Loan Scam,2161,9.010397e+06
16,Love/Parcel Scam,1821,2.032097e+07
25,S/M Impersonation Scam,1507,5.153877e+06


In [7]:

from datetime import timedelta, timezone
df_recovery_trend = df[['date_assigned', 'amount_scammed', 'latest_balance_seized']].copy()
df_recovery_trend = df_recovery_trend.fillna(0)

iteration_count = len(df_recovery_trend.index)
dates = []
amount_recovered_list = []
amount_recovered_interim = 0
start_date = df_recovery_trend['date_assigned'][0]
start_date = round(start_date.replace(tzinfo=timezone.utc).timestamp()) * 1000
current_date = start_date
end_date = df_recovery_trend['date_assigned'][iteration_count-1]

while current_date != end_date:
    dates.append(current_date)
    current_date += timedelta(days = 1)
dates.append(current_date)

current_date = start_date

for i in range(iteration_count):
    if df_recovery_trend['date_assigned'][i] == current_date:
        if df_recovery_trend['latest_balance_seized'][i] >= df_recovery_trend['amount_scammed'][i]:
            amount_recovered_interim += df_recovery_trend['amount_scammed'][i]
        else:
            amount_recovered_interim += df_recovery_trend['latest_balance_seized'][i]

    else:
        amount_recovered_interim = round(amount_recovered_interim, 2)
        amount_recovered_list.append(amount_recovered_interim)
        amount_recovered_interim = 0
        current_date += timedelta(days = 1)
        
        while df_recovery_trend['date_assigned'][i] != current_date:
            amount_recovered_list.append(0)   
            current_date += timedelta(days = 1)
            
        if df_recovery_trend['latest_balance_seized'][i] >= df_recovery_trend['amount_scammed'][i]:
            amount_recovered_interim += df_recovery_trend['amount_scammed'][i]
        else:
            amount_recovered_interim += df_recovery_trend['latest_balance_seized'][i]

amount_recovered_list.append(amount_recovered_interim)

print(len(amount_recovered_list))
print(len(dates))
print(dates)
print(amount_recovered_list)

447
447
[datetime.date(2022, 1, 1), datetime.date(2022, 1, 2), datetime.date(2022, 1, 3), datetime.date(2022, 1, 4), datetime.date(2022, 1, 5), datetime.date(2022, 1, 6), datetime.date(2022, 1, 7), datetime.date(2022, 1, 8), datetime.date(2022, 1, 9), datetime.date(2022, 1, 10), datetime.date(2022, 1, 11), datetime.date(2022, 1, 12), datetime.date(2022, 1, 13), datetime.date(2022, 1, 14), datetime.date(2022, 1, 15), datetime.date(2022, 1, 16), datetime.date(2022, 1, 17), datetime.date(2022, 1, 18), datetime.date(2022, 1, 19), datetime.date(2022, 1, 20), datetime.date(2022, 1, 21), datetime.date(2022, 1, 22), datetime.date(2022, 1, 23), datetime.date(2022, 1, 24), datetime.date(2022, 1, 25), datetime.date(2022, 1, 26), datetime.date(2022, 1, 27), datetime.date(2022, 1, 28), datetime.date(2022, 1, 29), datetime.date(2022, 1, 30), datetime.date(2022, 1, 31), datetime.date(2022, 2, 1), datetime.date(2022, 2, 2), datetime.date(2022, 2, 3), datetime.date(2022, 2, 4), datetime.date(2022, 2, 5

In [ ]:
#   x-min & x-max date (Default viewport for graph)                       
    max_date = date.today()
    time = datetime.min.time()
    default_max_datetime = datetime.combine(max_date, time)
    default_max_xview = round(default_max_datetime.replace(tzinfo=timezone.utc).timestamp()) * 1000

    min_date = max_date - timedelta(days = 90)
    default_min_datetime = datetime.combine(min_date, time)
    default_min_xview = round(default_min_datetime.replace(tzinfo=timezone.utc).timestamp()) * 1000
